In [1]:
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt
import math
import time
#相比于最开始拿到的版本，qiskit更新了几处表达，
#result.get_data（）改为result.data（），data的表达也有变化（可以print获得的data看看data不在表示为data['001']这样的形式）

In [2]:
# import math lib
from math import pi

# import Qiskit
from qiskit import Aer, execute#aer是模拟器
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# import basic plot tools
from qiskit.tools.visualization import plot_histogram

In [3]:
# To use local qasm simulator  模拟器改了 之前是qasm
backend = Aer.get_backend('aer_simulator')

In [4]:
def get_theta(d,m):
    i = 0
    theta = 0
    count = 0
    for i in d:
        count = i+count
   # print(d)
    theta = 2*math.acos(count/m)
   # print(theta)
    return theta

In [5]:
def get_Distance(x,y,m):
    theta_1 = get_theta(x,m)
    theta_2 = get_theta(y,m)
    
    # create Quantum Register called "qr" with 3 qubits
    qr = QuantumRegister(3, name="qr")
    # create Classical Register called "cr" with 1 bits
    cr = ClassicalRegister(1, name="cr")

    # Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
    # and your Classical Register "cr"
    qc = QuantumCircuit(qr, cr, name="k_means")
    
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    qc.u(theta_1, pi, pi, qr[1])
    qc.u(theta_2, pi, pi, qr[2])
    qc.cswap(qr[0], qr[1], qr[2])
    qc.h(qr[0])

    qc.measure(qr[0], cr[0])
    qc.reset(qr)

    #print('----before run----')
    job = execute(qc,backend=backend, shots=1024)
    #print('----after run----')
    result = job.result()
    data = result.data()['counts']
    #print(data)
    
    if len(data)==1:
        return 0.0
    else:
        return data['0x1']/1024.0

In [6]:
def get_data(n,k,std,m):     
    data = make_blobs(n_samples=n, n_features=m, centers=k, cluster_std=std, random_state=100)  
    points = data[0]   
    label = data[1]      
    return points,label

In [7]:
def get_distance(p1, p2):
    #与经典没变化
    return np.sqrt(np.sum((p1-p2)*(p1-p2)))

In [8]:
def find_nearest_neighbour(points,centroids,m):
    
    n = len(points)
    k = centroids.shape[0]
    centers = np.zeros(n)
    
    for i in range(n):
        min_dis = 10000
        ind = 0#（Internodal Distance 节点间距离？）
        for j in range(k):
            #若这里为get_distance(points[i,:],centroids[j,:])则为经典算法
            #这里只对最近邻计算做量子化处理。是否还有可能用量子计算机处理的地方？
            
            temp_dis = get_Distance(points[i,:],centroids[j,:],m)
            
            if temp_dis < min_dis:
                min_dis = temp_dis
                ind = j
        centers[i] = ind
    
    return centers

In [9]:
def find_centroids(points,centers):
    n = len(points)
    k = int(np.max(centers))+1
   
    centroids = np.zeros([k,384])
    
    for i in range(k):
        centroids[i,:] = np.average(points[centers==i])
        
    return centroids

In [10]:
def preprocess(points):    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(points)  
    
    return normalized_data

In [11]:
def initialize_centers(points,k):  
    return points[np.random.randint(points.shape[0],size=k),:]

In [12]:
t1 = 100
t2 = 3 
output = np.array([['v_measure','标签种类(k)']])
for n1 in range(t1):  
    
    n = 100
    data_num=n# number of data points 
    k = 5     # Number of centers 
    std = 2      # std of data points 
    m = 384
    points,y_true = get_data(n,k,std,m)    
    #dataset
    #print(points) 
    points = preprocess(points)# Normalize dataset 
    #print(points) 
    centroids1 = initialize_centers(points,k) 
    start_time = time.time() 
    # run k-means algorithm 
    for i in range(t2):  
        centers = find_nearest_neighbour(points,centroids1,m)  # find nearest centers  
        centroids = find_centroids(points,centers)               # find centroids    
    end_time = time.time() 
    cost_time = end_time - start_time 
    print(cost_time)
    from sklearn.metrics import v_measure_score
    y_pred = centers 
    result = v_measure_score(y_true, y_pred)
    output = np.insert(output,n1+1,[str(result),str(k)],axis=0)  
output = np.insert(output,0,['第'+str(n1+1)+'次',str(data_num)],axis=0)
        
filename = '%s组重复 第%s次.csv'%(t1,n1+1)  
np.savetxt(filename,output,fmt='%s',delimiter=',')   

    

14.945702314376831
14.768010139465332


d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


14.712686538696289


d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


14.703598499298096
14.77123236656189
14.798415184020996
14.751136064529419
14.68720293045044


...

14.866966247558594


d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


14.818188190460205
14.952209234237671
14.81845474243164
14.803894519805908
14.86898922920227
14.84048867225647


d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


14.851441383361816
